# Embeddings No Contextuales con FastText

En este notebook se generan embeddings no contextuales para las noticias utilizando FastText. Los embeddings se obtienen promediando los vectores de palabras de cada texto.

Importar todas las librerías necesarias

In [1]:
import fasttext.util
import pandas as pd
import os
import numpy as np
from nltk.tokenize import word_tokenize
import nltk

Descargar recursos necesarios de NLTK

In [2]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /home/adiez/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Definir directorios de donde queremos cargar los datos resumidos y donde queremos guardar los embeddings

In [3]:
# Usa el directorio actual del notebook como raíz del proyecto
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Define las rutas a los directorios de datos
SUMMARIES_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'summarized')
FASTTEXT_PATH = os.path.join(ROOT_DIR, 'models', 'cc.en.300.bin')
OUTPUT_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'embeddings')

# Crea el directorio de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Lista todos los archivos .parquet en el directorio SUMMARIES_DIR
parquet_files = [f for f in os.listdir(SUMMARIES_DIR) if f.endswith('.parquet')]

# Muestra el número total de archivos encontrados
print(f"Se han encontrado {len(parquet_files)} archivos parquet para procesar.\n")

# Lista los nombres de los archivos encontrados
for file in parquet_files:
    print(file)

Se han encontrado 7 archivos parquet para procesar.

amazon_news.parquet
apple_news.parquet
google_news.parquet
meta_news.parquet
microsoft_news.parquet
nvidia_news.parquet
tesla_news.parquet


Cargar modelo preentrenado de FastText

In [4]:
# Cargar modelo FastText
ft = fasttext.load_model(FASTTEXT_PATH)
print("Modelo FastText cargado.")
print(f"Dimensión de embeddings: {ft.get_dimension()}")

Modelo FastText cargado.
Dimensión de embeddings: 300


Definir función para generar embeddings de texto promediando vectores de palabras

In [5]:
def get_text_embedding(tokens):
    """
    Genera embedding de texto promediando los vectores de palabras.
    """
    embeddings = []
    for token in tokens:
        embeddings.append(ft.get_word_vector(token))
    
    if len(embeddings) == 0:
        return np.zeros(ft.get_dimension())
    
    return np.mean(embeddings, axis=0)

Procesamiento de archivos: para cada archivo se realiza la carga de datos, concatenación de título y resumen, tokenización, generación de embeddings y guardado de resultados

In [6]:
for file_name in parquet_files:
    print(f"Procesando {file_name}...")
    
    # Cargar dataset
    df = pd.read_parquet(os.path.join(SUMMARIES_DIR, file_name), engine="pyarrow")
    
    # Concatenar título y resumen
    df["text"] = df["title"] + "\n\n" + df["body_summary"]
    df = df.drop(columns=["clean_body", "body_summary", "teaser", "title", "author"])
    
    # Eliminar filas con texto vacío
    df = df.dropna(subset=['text'])
    
    # Tokenizar la columna 'text'
    df['tokens'] = df['text'].apply(lambda x: word_tokenize(x.lower()))
    
    # Generar embeddings
    df['embedding'] = df['tokens'].apply(get_text_embedding)
    
    # Eliminar columna de tokens
    df = df.drop(columns=['tokens'])
    
    # Guardar en directorio de salida
    output_name = file_name.replace('_news.parquet', '_embeddings_no_context.parquet')
    output_path = os.path.join(OUTPUT_DIR, output_name)
    df.to_parquet(output_path, engine="pyarrow", index=False)
    
    print(f"  Guardado {output_name} con {len(df)} registros y dimensión de embedding {ft.get_dimension()}\n")

print("Todos los archivos han sido procesados exitosamente!")
print("Archivos guardados en:", OUTPUT_DIR)

Procesando amazon_news.parquet...
  Guardado amazon_embeddings_no_context.parquet con 12391 registros y dimensión de embedding 300

Procesando apple_news.parquet...
  Guardado amazon_embeddings_no_context.parquet con 12391 registros y dimensión de embedding 300

Procesando apple_news.parquet...
  Guardado apple_embeddings_no_context.parquet con 24208 registros y dimensión de embedding 300

Procesando google_news.parquet...
  Guardado apple_embeddings_no_context.parquet con 24208 registros y dimensión de embedding 300

Procesando google_news.parquet...
  Guardado google_embeddings_no_context.parquet con 13341 registros y dimensión de embedding 300

Procesando meta_news.parquet...
  Guardado google_embeddings_no_context.parquet con 13341 registros y dimensión de embedding 300

Procesando meta_news.parquet...
  Guardado meta_embeddings_no_context.parquet con 15187 registros y dimensión de embedding 300

Procesando microsoft_news.parquet...
  Guardado meta_embeddings_no_context.parquet con